In [ ]:
 #!pip3 install keras imgaug
 #!pip install keras_preprocessing

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image, ImageOps
import tensorflow


import numpy as np

import time
import copy

#tensorflow.keras is the neural network library from Tensorflow
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras import Model, layers
from tensorflow.keras.models import load_model, model_from_json
from tensorflow.keras.optimizers.legacy import Adam

import json



In [ ]:
import os
import io

WORKING_DIRECTORY = '/'

os.chdir(WORKING_DIRECTORY)

In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)


TensorFlow version: 2.14.0


In [ ]:
import tensorflow as tf
print("cuDNN version used by TensorFlow:", tf.config.experimental.list_physical_devices('GPU'))


cuDNN version used by TensorFlow: []


In [ ]:
labels = ["E00-0","E0A-0","EOF-C","E1B-A","E1D-8","E2C-8","E3A-1","E04-2","E4E-5","E05-3","E5A-A","E5C-3","E06-2","E6B-2","E6E-0","E16-3","E18-B","E22-3","E24-4","E27-7","E38-0","E39-6","E38-0","E39-6","E46-D","E51-1","E53-B","E60-D","E68-2","E70-4","Other"];
train_datagen = ImageDataGenerator(
        rotation_range=45,
        width_shift_range=0.2,
        height_shift_range=0.2,
        brightness_range=[0.2,1.0],
        shear_range=0.1,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    # train_data_dir,
    '#Change to data path',
    batch_size=32, #batch size of
    class_mode='categorical', ##more than 2 classification
    target_size=(224,224)) ##resnet target size


validation_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input)

validation_generator = validation_datagen.flow_from_directory(
    # val_data_dir,
    '#Change to data path',
    shuffle=False,
    batch_size = 32,
    class_mode='categorical',
    target_size=(224,224))


Found 671 images belonging to 31 classes.
Found 162 images belonging to 31 classes.


In [ ]:

conv_base = ResNet50(include_top=False,weights='imagenet') ##using resnet 50

for layer in conv_base.layers: #only train top/last layer
    layer.trainable = False


x = conv_base.output ##NN output layer
x = layers.GlobalAveragePooling2D()(x) ##use GlobalAveragePooling2D
x = layers.Dense(128, activation='relu')(x) #relu activation
predictions = layers.Dense(31, activation='softmax')(x) #softmax prediction
model = Model(conv_base.input, predictions)

optimizer = tensorflow.keras.optimizers.Adam() ##adam optimizer
model.compile(loss='categorical_crossentropy',  ##trying crossentropy loss
              optimizer=optimizer,
              metrics=['accuracy'])

In [ ]:
#get summary of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None, None, 3)]      0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, None, None, 3)        0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, None, None, 64)       9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, None, None, 64)       256       ['conv1_conv[0][0]']          
 on)                                                                                          

In [ ]:
#do not run this cell unless training
history_data = model.fit(
    train_generator,
    epochs=100,
    validation_data=validation_generator)

Epoch 1/100
21/21 [==============================] - 10s 478ms/step - loss: 1.9078 - accuracy: 0.4709 - val_loss: 1.8246 - val_accuracy: 0.4568
Epoch 2/100
21/21 [==============================] - 10s 478ms/step - loss: 1.7436 - accuracy: 0.4873 - val_loss: 1.9100 - val_accuracy: 0.4444
Epoch 3/100
21/21 [==============================] - 10s 477ms/step - loss: 1.7130 - accuracy: 0.5171 - val_loss: 1.6824 - val_accuracy: 0.4938
Epoch 4/100
21/21 [==============================] - 10s 477ms/step - loss: 1.6023 - accuracy: 0.5395 - val_loss: 1.4871 - val_accuracy: 0.5494
Epoch 5/100
21/21 [==============================] - 10s 477ms/step - loss: 1.5185 - accuracy: 0.5455 - val_loss: 1.4354 - val_accuracy: 0.5802
Epoch 6/100
21/21 [==============================] - 10s 482ms/step - loss: 1.4828 - accuracy: 0.5723 - val_loss: 1.4665 - val_accuracy: 0.5617
Epoch 7/100
21/21 [==============================] - 10s 477ms/step - loss: 1.3619 - accuracy: 0.6006 - val_loss: 1.3772 - val_accuracy:

In [ ]:
#Save the model
model.save('#Change to model path/model.h5')

In [ ]:
#plot the history for accuracy during training
plt.plot(history_data['accuracy'])
plt.plot(history_data['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validate'], loc='upper left')
plt.show()

#plot for loss
plt.plot(history_data['loss'])
plt.plot(history_data['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validate'], loc='upper left')
plt.show()

In [ ]:
#load the model
model = load_model('#Change to model path')

#dictionary for labels
labels = train_generator.class_indices
labels = {value:key for key, value in labels.items()}
labels

In [ ]:
from keras_preprocessing import image
#load image

def test(img, labels):

  #img to array
  img_array = image.img_to_array(img)

  img_batch = np.expand_dims(img_array, axis = 0)

  #preprocess it
  img_prepocessed = preprocess_input(img_batch)

  #softmax gives probability distribution
  pred = model.predict(img_prepocessed)

  #get highest value
  pred_classes = pred.argmax(axis=-1)

  print(pred[0])
  print(pred[0][pred_classes])
  print(labels[pred_classes[0]])
  plt.imshow(img)

